In [2]:
#Import the required dataset
data = pd.read_excel("Telco_customer_churn.xlsx")
demo = pd.read_excel("Telco_customer_churn_demographics copy.xlsx")
serv = pd.read_excel("Telco_customer_churn_services.xlsx")

#Renaming, merging, and dropping columns
data = data.rename(columns = {"CustomerID" : "Customer ID"})
data = pd.merge(data, demo[['Customer ID','Age', 'Married']], on = "Customer ID")
data = pd.merge(data, serv[['Customer ID', 'Number of Referrals', 'Avg Monthly GB Download', 'Streaming Music', 'Unlimited Data', 'Total Refunds', 'Total Extra Data Charges', 'Total Long Distance Charges']], on = 'Customer ID')
data = data.drop(columns = ["City", "Lat Long", "Count", "Country", "State", "Churn Reason", 'Total Charges', 'Churn Label'])
data = pd.merge(data, serv[['Customer ID','Total Charges', 'Total Revenue']], on = "Customer ID")
data = data.drop(columns = ['Customer ID'])

#Re-arranging the columns
cols = list(data.columns.values)
cols = ['Zip Code','Latitude','Longitude','Gender','Senior Citizen','Partner','Dependents','Tenure Months',
 'Phone Service','Multiple Lines','Internet Service','Online Security','Online Backup','Device Protection',
 'Tech Support','Streaming TV','Streaming Movies','Contract','Paperless Billing','Payment Method','Monthly Charges',
 'Churn Score','CLTV','Age','Married','Number of Referrals','Avg Monthly GB Download','Streaming Music',
 'Unlimited Data','Total Refunds','Total Extra Data Charges','Total Long Distance Charges','Total Charges',
 'Total Revenue','Churn Value']
data = data[cols]

In [3]:
#Split features from target variable
x = data.iloc[:, :34]
y = data.iloc[: , -1]

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    stratify=y, 
                                                    test_size=0.20, random_state = 50)

In [5]:
y_train.value_counts()

0    4139
1    1495
Name: Churn Value, dtype: int64

In [6]:
oversample = SMOTENC(categorical_features = [3,4,5,6,8,9,10,11,12,13,14,15,16,17,18,19,24,27,28])
x_train, y_train = oversample.fit_resample(x_train, y_train)

In [7]:
y_train.value_counts()

0    4139
1    4139
Name: Churn Value, dtype: int64

In [8]:
x_train.to_csv(r'x_train_q2.csv', index = False, header = True)
x_test.to_csv(r'x_test_q2.csv', index = False, header = True)

In [9]:
#Train sets (x)
FAMD_train_16 = pd.read_excel("FAMD_train_16_q2.xlsx")

#Test sets (x)
FAMD_test_16 = pd.read_excel("FAMD_test_16_q2.xlsx")

In [10]:
range_n_clusters = list (range(2,9))
for n_clusters in range_n_clusters:
    clusterer = KMeans(n_clusters=n_clusters)
    preds = clusterer.fit_predict(FAMD_test_16)
    centers = clusterer.cluster_centers_
    score = silhouette_score(FAMD_test_16, preds)
    print("For n_clusters = {}, silhouette score is {})".format(n_clusters, score))

For n_clusters = 2, silhouette score is 0.29076906208469033)
For n_clusters = 3, silhouette score is 0.22751900600825195)
For n_clusters = 4, silhouette score is 0.2329947739959302)
For n_clusters = 5, silhouette score is 0.18400038508876798)
For n_clusters = 6, silhouette score is 0.17625507370743165)
For n_clusters = 7, silhouette score is 0.13001012103225323)
For n_clusters = 8, silhouette score is 0.1274789233167387)


In [11]:
kmeans = KMeans(n_clusters=2)
train_clusters = kmeans.fit(FAMD_train_16)
test_clusters = train_clusters.predict(FAMD_test_16)

In [12]:
FAMD_train_16['cluster'] = train_clusters.labels_
FAMD_test_16['cluster'] = test_clusters

# SVM hybrid

In [13]:
svm_model = svm.SVC()
kernel = ['poly', 'rbf', 'sigmoid']
C = [50, 10, 1.0, 0.1, 0.01]

# define grid search
grid = dict(kernel=kernel,C=C)
cv = StratifiedKFold(n_splits=10, shuffle=True)
grid_search = GridSearchCV(estimator=svm_model, param_grid=grid, n_jobs=-1, cv=cv, scoring='f1',error_score=0)
grid_result = grid_search.fit(FAMD_train_16, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.923582 using {'C': 10, 'kernel': 'rbf'}
0.913604 (0.010554) with: {'C': 50, 'kernel': 'poly'}
0.916463 (0.008444) with: {'C': 50, 'kernel': 'rbf'}
0.840643 (0.009336) with: {'C': 50, 'kernel': 'sigmoid'}
0.915353 (0.009372) with: {'C': 10, 'kernel': 'poly'}
0.923582 (0.008140) with: {'C': 10, 'kernel': 'rbf'}
0.840842 (0.009197) with: {'C': 10, 'kernel': 'sigmoid'}
0.911683 (0.009780) with: {'C': 1.0, 'kernel': 'poly'}
0.921107 (0.009782) with: {'C': 1.0, 'kernel': 'rbf'}
0.848384 (0.013269) with: {'C': 1.0, 'kernel': 'sigmoid'}
0.885264 (0.011709) with: {'C': 0.1, 'kernel': 'poly'}
0.903410 (0.010548) with: {'C': 0.1, 'kernel': 'rbf'}
0.879655 (0.012221) with: {'C': 0.1, 'kernel': 'sigmoid'}
0.806736 (0.010507) with: {'C': 0.01, 'kernel': 'poly'}
0.860316 (0.010104) with: {'C': 0.01, 'kernel': 'rbf'}
0.878492 (0.012191) with: {'C': 0.01, 'kernel': 'sigmoid'}


In [14]:
svm_model = svm.SVC(C = 10, kernel = "rbf")
svm_model.fit(FAMD_train_16, y_train)
y_pred = svm_model.predict(FAMD_test_16)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.61      0.71      1035
           1       0.39      0.68      0.50       374

    accuracy                           0.63      1409
   macro avg       0.62      0.65      0.60      1409
weighted avg       0.72      0.63      0.65      1409



# KNN hybrid

In [15]:
# define models and parameters
knn_model = KNeighborsClassifier()
n_neighbors = range(1, 21, 2)
weights = ['uniform', 'distance']
metric = ['euclidean', 'manhattan', 'minkowski']
# define grid search
grid = dict(n_neighbors=n_neighbors,weights=weights,metric=metric)
cv = StratifiedKFold(n_splits=10, shuffle=True)
grid_search = GridSearchCV(estimator=knn_model, param_grid=grid, n_jobs=-1, cv=cv, scoring='f1',error_score=0)
grid_result = grid_search.fit(FAMD_train_16, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.893160 using {'metric': 'euclidean', 'n_neighbors': 7, 'weights': 'distance'}
0.865706 (0.010121) with: {'metric': 'euclidean', 'n_neighbors': 1, 'weights': 'uniform'}
0.865706 (0.010121) with: {'metric': 'euclidean', 'n_neighbors': 1, 'weights': 'distance'}
0.884142 (0.009733) with: {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'uniform'}
0.885599 (0.008367) with: {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'}
0.888001 (0.008065) with: {'metric': 'euclidean', 'n_neighbors': 5, 'weights': 'uniform'}
0.890179 (0.008393) with: {'metric': 'euclidean', 'n_neighbors': 5, 'weights': 'distance'}
0.890580 (0.008795) with: {'metric': 'euclidean', 'n_neighbors': 7, 'weights': 'uniform'}
0.893160 (0.009505) with: {'metric': 'euclidean', 'n_neighbors': 7, 'weights': 'distance'}
0.890140 (0.009792) with: {'metric': 'euclidean', 'n_neighbors': 9, 'weights': 'uniform'}
0.892566 (0.010578) with: {'metric': 'euclidean', 'n_neighbors': 9, 'weights': 'distance'}
0.888906 

In [16]:
knn_model = KNeighborsClassifier(metric = "euclidean", n_neighbors = 7, weights = "distance")
knn_model.fit(FAMD_train_16, y_train)
y_pred = knn_model.predict(FAMD_test_16)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.59      0.71      1035
           1       0.42      0.84      0.56       374

    accuracy                           0.65      1409
   macro avg       0.67      0.71      0.64      1409
weighted avg       0.78      0.65      0.67      1409

